In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install sentence_transformers
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sentence_transformers import SentenceTransformer

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
%cd "./drive/My Drive"

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

### Loading data (temporary)

In [ ]:
df = pd.read_csv("./processed_train_data.csv")
X = df["Expert"].values
# y = Load in labels here

# Or use/create DataLoaders

### Helper functions (temporary)

In [ ]:
def get_logits(model, X, tokenizer):
  # Tokenize the input text
  inputs = tokenizer.encode("$simple$ ; $expert$ = " + X, return_tensors="pt").to(device)

  with torch.no_grad():
    logits = model(inputs, decoder_input_ids=inputs).logits

  return (logits.squeeze())

In [ ]:
def make_label(model, X, tokenizer):
  model.eval()
  # Tokenize the input text
  inputs = tokenizer.encode("$simple$ ; $expert$ = " + X, return_tensors="pt")

  # Generate output from the model
  outputs = model.generate(inputs.to(device), num_beams=4, early_stopping=True, max_new_tokens=200)

  # Decode the output tokens to text
  output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  return (output_text)

In [ ]:
def train(student,
          X,
          y,
          loss_fn,
          optimizer):

  student.train()
  train_loss = 0

  for i in range(len(X)):
    y_student = get_logits(student, X[i], tokenizer)

    loss = loss_fn(y_student, y[i].requires_grad_())
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss = train_loss / len(X)
  return(train_loss)

### Model

In [ ]:
student = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
tokenizer = T5Tokenizer.from_pretrained("t5-small")

loss_fn = nn.CrossEntropyLoss();
optimizer = torch.optim.Adam(student.parameters())

### Train (temporary)

In [ ]:
# loss_history = []
for epoch in range(1000):
  loss = train(student, X[:80], logits[:80], loss_fn, optimizer)
  # loss_history.append(loss)
  # torch.save({
  #     "epoch": epoch,
  #     "model_state_dict": student.state_dict(),
  #     "optimizer_state_dict": optimizer.state_dict(),
  #     "train_loss_history": loss_history
  # }, "./student_actual.pth")
  if (epoch % 10 == 0):
    print(loss)